# [Worst-Case to Average-Case reduction based on Gaussian Measure](https://cims.nyu.edu/~regev/papers/average.pdf)
Micciancio and Regev 2005

## Introduction

Assume that you are dealing with the computational task of factoring a 100,000-bit composite number $N$ into _any two_ numbers of smaller size. If $N$ is even, then $(2, \frac{N}{2})$ is a valid answer. The complexity of computing this result is $1$ bit-wise $\wedge$ operation followed by one rotate-right ($\gg$) operation. Both these operations can be performed essentially in $O(1)$ time using appropriate data structures. On the other hand, if $N$ is odd, then the complexity of factoring, roughly speaking, depends on the length of the smallest factor (ignoring number field seive for the moment).

So even though our task is to factor 100,000-bit numbers, the concrete time complexity of the problem is heavily dependent on the actual input instance of the problem. In such a scenario, making meaningful _asymptotic comparison_ between two different algorithms that sove the same problem, it's advantageous to define a metric over _all instances_ of the problem of a given input length. This metric can be defined in different ways to capture different computational aspect of the problem, just like $\ell_1$, $\ell_2$, $\ell_{\infty}$ norm are defined to capture different geometric properties of a set.

The most prevelant metric used in computational complexity is that of worst-case complexity. The **worst-case complexity** is the upper bound on the resource consumption (time, space) of the algorithm, where the upper bound is computed over _all_ $\lambda$-bit input instances of the problem. For example, if we conjecture that numbers that are product of two equal-length primes are hardest to factor, then the worst-case complexity will only capture the difficulty of factoring such numbers. Numbers that are even, that require $O(1)$ time to factor, will be conveniently ignored. Even among numbers that are product of two primes, there will be few instances that are hardest among the hard (i.e., consider the equivalence complexity class $O(\lambda)$, $O(\lambda^2)$, $\cdots$, $2^{\tilde{O}(\lambda)}$ of each instance, ignoring constant terms). The worst-case complexity essentially captures the resource requirements of such isolated instances. In practice, it's entirely possible (SAT  is a good example) that only a statistically negligible fraction of problems are hard, while the rest that actually do arise in practice are easy.

In cryptography, however, we would like the assurance that with high probability, _every_ $\lambda$-bit concrete instance of the problem that we select is hard. The worst-case complexity is not a good metric for this. What we need are problem instances that are hard _on average_. 

In order to talk about averages, we need to define a probability distribution on how the problems are chosen. This leads to the notion of **distributional problems**, which not only specifies what the algorithm will compute (e.g., factor a number), but also the probability distribution $\mu_{\lambda}$ from which the $\lambda$-bit instances will be selected. For example, in case of factoring, if $\mu_{\lambda}$ is uniform (i.e., every $\lambda$-bit number is equally likely as input), then roughly half the problem-instances (all even numbers) will have very low complexity. On the other hand, one could insist that $\mu_{\lambda}$ be the distribution of those numbers that are product of two equal length primes. In this case, the average complexity, assuming heuristically that such numbers are hardest to solve, will be much higher. For more details on Average Case Complexity, see the [excellent survey](https://arxiv.org/abs/cs/0606037) by Andrej Bogdanov and Luca Trevisan.

Distributional problems are a good way to understand the average case complexity of an algorithm, however, in practice there are two problems with it: First, for most real life problems, it's not easy to determine $\mu_\lambda$ that corresponds to the hardest problems. In case of factoring, do we know the distribution of hardest factoring problems? Second, even if we know the distribution $\mu_\lambda$, we might still not know how to efficiently sample (i.e., in polynomial-time) problem-instances from this distribution. It could be that sampling from such a distribution requires an exhaustive search over the entire $\lambda$-bit input string. For example, to make factoring as hard as possible, one could enumerate all $\lambda$-bit integer, factor all of them using the algorithm at hand, and only select the hardest to factor numbers from this list! This procedure is clearly not efficiently-samplable!

To build cryptographic protocols based on strong theoretical foundations, we need _efficiently-samplable_ problems that are guaranteed to be hard on average. The final cherry on the cake is when these average-case problems can be shown to be as hard as certain well-known worst-case problems. A remarkable thing about Lattices--in spite of the boring Linear Algebra feel to it--provides such a connection.

Ajtai's [seminal paper](https://eccc.weizmann.ac.il/eccc-reports/1996/TR96-007/Paper.pdf) in 1995 described such a worst-case to average-case connection between Short Independent Vector Problem (SIVP) and Short Integer Solution (SIS). Ajtai's original reduction is a bit cumbersome (and was hard for me to understand), but Micciancio and Regev greatly simplified it using Discrete Gaussian Sampling. The rest of this notebook is about Micciancio and Regev 2005 reduction.

## Methedology for Reductions in Code

Micciancio and Regev paper is very well written and relatively easy to understand, however, in order to run these reductions as a computer program one needs to implement the oracle that solves the hard problem. But since the problem is hard-to-solve there's little hope to run these reductions on cryptographically relevant parameter sizes. So what should one do?

Suppose algorithm $\mathcal{A}(x_1,x_2, \cdots, x_p)$, reduces to algorithm $\mathcal{B}(y_1, y_2,\cdots,y_q)$, where $\mathcal{B}$ is the Oracle that solves the hard problem. The approach taken in this notebook is that $\mathcal{B}$ is implemented in such a way that it hands out problem instances for $\mathcal{A}$ such that when $\mathcal{A}$ calls $\mathcal{B}$, $\mathcal{B}$ will know how to solve it! This may seem completely backwards (and it is), but this allows maximum flexibility in terms of understanding the reduction. Furthermore, for Gap problems, it allows $\mathcal{B}$ to return wrong results, not terminate if the problem instance is within the Gap etc.

### Distinguishers and Cook/Turing Reductions

In many cryptographic schemes, an oracle (algorithm $\mathcal{B}$) acts as a distinguisher between two different distributions. It is my belief that Cook/Turing reductions are ill suited for this task, even though Cook reductions are everywhere in cryptography. 

Recall that in a Cook style reduction, algorithm $\mathcal{A}$ is allowed to make multiple _independent_ calls to $\mathcal{B}$ and output its own result by combining the response from $\mathcal{B}$ in some clever way. The difficulty with distinguishers is that one needs more than one sample to decide the distribution of the input. For example, assume that there's a distinguisher that can decide between Uniform and Gaussian distributions over $[0,1)$. (This problem is trivial.) However, if the oracle is fed one sample at a time, say $0.4$, then $0.5$, then $0.5$, and finally $0.1$, the distinguisher cannot decide if the first $0.4$ came from Uniform or Gaussian -- The oracle just doesn't have sufficient state to make a judgement. While this might be an extreme example, Cook reductions require a createful design and conditional probability kung-fu when implemented in code. (**NOTE**: Karp  reductions don't suffer from this problem, because $[0.4,0.5,0.5,0.1]$ will be fed to oracle at once, and it can then esily decide the distribution. However, most cryptographic reductions don't use Karp reductions.)

## Prelimnaries


### Lattices

A _lattice_ $\Lambda$ is a set of vectors that's generated by all integer linear combination of $n$ linearly independent vectors $\{\mathbf{b}_1, \mathbf{b}_2, \cdots, \mathbf{b}_n \}$ in $\mathbb{R}^n$, i.e.,

$
\Lambda(\mathbf{b}_1,\mathbf{b}_2,\cdots,\mathbf{b}_n) = \left \{ \sum_{i=0}^n z_i \mathbf{b}_i : z_i \in \mathbb{Z}, \mathbf{b}_i \in \mathbb{R}^n \right \}
$

The basis $\{ \mathbf{b}_i \}$ can be represented as a matrix $\mathbf{B} = [\mathbf{b}_1, \mathbf{b}_2, \cdots, \mathbf{b}_n] \in \mathbb{R}^{n\times n}$ where each $\mathbf{b}_i$ is a column vector and $\Lambda(\mathbf{B}) = \mathbf{B}\mathbf{z}$ where $\mathbf{z} \in \mathbb{Z}^n$.

The _fundamental parallelepiped_ $\mathcal{P}(\mathbf{B})$ of a lattice is defined as 

$$
\mathcal{P}(\mathbf{B}) := \{ \mathbf{B}\mathbf{x} |  \mathbf{x} \in \mathbb{R}^n \cap [0,1)^n \}
$$

The _determinant_ of a Lattice is the volume of $\mathcal{P}(\mathbf{B})$ which is equal to the determinant of $\mathbf{B}$.

The length of $n$ shortest linearly-independent non-zero vectors in the lattice is denoted by $\lambda_1(\Lambda) \leq \lambda_2(\Lambda) \leq \cdots \leq \lambda_n(\Lambda)$. An application of [Minkowski Convex Body theorem](http://math.uga.edu/~pete/4400Minkowski.pdf) implies that

$$
\lambda_1(\Lambda) \leq \sqrt{n}\det(\Lambda)^{\frac{1}{n}}
$$

### Dual Lattice

The dual of a Lattice $\Lambda$ is the set 

$$
\Lambda^* := \left \{\mathbf{x}: \forall \mathbf{y} \in \Lambda, \langle \mathbf{x}, \mathbf{y} \rangle \in \mathbb{Z} \right \}
$$

(The dual lattice is also sometimes called **reciprocal lattice** or **polar lattice**.)

The dual of a Lattice is also a lattice (proof: routine). Since the lattice itself is an integer linear combination of basis vector, the inner product of the dual lattice with the basis vectors in primal lattice must be an integer. This can be used to efficiently compute dual basis of a Lattice.

**Example**: Let $\Lambda = \mathbb{Z}\begin{bmatrix}{2\\1}\end{bmatrix} + \mathbb{Z}\begin{bmatrix}{1\\3}\end{bmatrix} \in \mathbb{Z}^2$. To compute $\Lambda^*$, let $\begin{bmatrix}{a\\b}\end{bmatrix} \in \Lambda^*$. This is equivalent to saying $\left \langle \begin{bmatrix}{2\\1}\end{bmatrix}, \begin{bmatrix}{a\\b}\end{bmatrix} \right \rangle = 2a + b \in \mathbb{Z}$ and $ \left \langle \begin{bmatrix}{1\\3}\end{bmatrix}, \begin{bmatrix}{a\\b}\end{bmatrix} \right \rangle = a + 3b \in \mathbb{Z}$. 

Equivalently, $\forall x,y \in \mathbb{Z}$

$$
\begin{matrix}
2a + b & = & x \\
a + 3b & = & y \\
\end{matrix}
$$

which is equivalent to 

$$
\begin{matrix}
a & = & \frac{3}{5}x - \frac{1}{5} y\\
b & = & -\frac{1}{5}x + \frac{2}{5} y\\
\end{matrix}
$$

or $\Lambda^* = \frac{1}{5}\left (\mathbb{Z}\begin{bmatrix}{3\\1}\end{bmatrix} + \mathbb{Z}\begin{bmatrix}{-1\\2}\end{bmatrix} \right )$

**Definition**: For a basis $\mathbf{B}=[\mathbf{b}_1,\mathbf{b}_2,\cdots,\mathbf{b}_n]$, _define_ the *dual basis* $\mathbf{D} = [\mathbf{d}_1,\mathbf{d}_2,\cdots,\mathbf{d}_n]$ as the unique basis that satisfies:

1. $\text{span}_{\mathbb{R}}(\mathbf{B}) = \text{span}_{\mathbb{R}}(\mathbf{D})$
2. $\mathbf{B}^T\mathbf{D} = \mathbf{I}_n$

Working as in the example above, the second condition can be interpreted as $\langle \mathbf{b}_i, \mathbf{d}_j \rangle = \delta_{i,j}$, which gives a standard formula for computing $\mathbf{D} = \mathbf{B}(\mathbf{B}^T\mathbf{B})^{-1}$. (This proof is not just sloppy, it's wrong!)

**Lemma** : For Lattices $\Lambda, \Lambda_1, \Lambda_2$ in $\mathbb{R}^n$, the following properties hold:

1. $(\Lambda^*)^* = \Lambda$
2. $ \Lambda_1 \subset \Lambda_2 \Leftrightarrow \Lambda_2^* \subset \Lambda_1^*$
3. $ (\Lambda_1 + \Lambda_2)^* = \Lambda_1^* \cap \Lambda_2^*$
4. $ (\Lambda_1 \cap \Lambda_2)^* = \Lambda_1^* + \Lambda_2^*$
5. $ \frac{1}{\det(\Lambda)} = \det{\Lambda^*}$

**Proof**: 1-4: Routine. 5 (sloopy): By condition 2 in the definition of dual basis, $\mathbf{B}^T\mathbf{D}=\mathbf{I}_n$. Taking determinants, we get $\det(\mathbf{B}^T\mathbf{D})= \det(\mathbf{B}^T)\det(\mathbf{D})= 1 = \det(\Lambda)\det(\Lambda^*)$.

**Lemma**: For a full rank lattice $\Lambda \subset \mathbb{R}^n$ and any (well behaved) function $f : \mathbb{R}^n \rightarrow \mathbb{C}$, 

$$
f(\Lambda) := \sum_{\mathbf{x} \in \Lambda}f(\mathbf{x}) = \det(\Lambda^*)\sum_{\mathbf{y}\in \Lambda^*}\hat{f}(\mathbf{y}) = \det(\Lambda^*) \hat{f}(\Lambda^*) 
$$

where $\hat{f}$ is the fourier transform of $f$, and $\Lambda^*$ is the dual lattice.

**Proof**: Let $\phi(x) = \sum_{z \in \Lambda}f(x+z)$. For any $y \in \Lambda$, by definition $\phi(x) = \phi(x+y)$, therefore $\phi(x)$ is a $\Lambda$-period function. Consider the Fourier series of $\phi$



$$ 
\begin{aligned}
\hat{\phi}(y) & = \frac{1}{\det(\Lambda)}\int_{\mathcal{P}(\mathbf{B})} \sum_{z\in\Lambda}f(x+z)e^{-2\pi i \langle x, y \rangle} dx \\
 & = \frac{1}{\det(\Lambda)} \sum_{z\in\Lambda}\int_{\mathcal{P}(\mathbf{B})} f(x+z)e^{-2\pi i \langle x, y \rangle} dx \\
 & = \frac{1}{\det(\Lambda)} \sum_{z\in\Lambda}\int_{\mathcal{P}(\mathbf{B})} f(x+z)e^{-2\pi i \langle x+z, y \rangle} dx  \\
 & = \frac{1}{\det(\Lambda)} \int_{\mathbb{R}^n} f(x)e^{-2\pi i \langle x, y \rangle} dx \\
 & = \det(\Lambda^*) \int_{\mathbb{R}^n} f(x)e^{-2\pi i \langle x, y \rangle} dx \\
 & = \det(\Lambda^*) \hat{f}(y)
\end{aligned}
$$

Now $\phi(0) = \sum_{z \in \Lambda}f(z) = f(\Lambda)$ and by inverse Fourier transform $\phi(0) = \sum_{y \in \Lambda^*}\hat{\phi}(y) = \det(\Lambda^*) \sum_{y \in \Lambda^*}\hat{f}(y) = \det(\Lambda^*)\hat{f}(\Lambda^*)$

Therefore, $f(\Lambda) = \det(\Lambda^*)\hat{f}(\Lambda^*)$.

### Banaszczyk Transference Theorems

**Baby Transference Theorem** : $\lambda_1(\Lambda)\lambda_1(\Lambda^*) \leq n$. 

**Proof**: By Minkowski theorem for any full-rank Lattice $\Lambda$, 

$$
\lambda_1(\Lambda) \leq \sqrt{n}\det(\Lambda)^{\frac{1}{n}}
$$

and the same holds for the dual lattice $\Lambda^*$

$$
\lambda_1(\Lambda^*) \leq \sqrt{n}\det(\Lambda^*)^{\frac{1}{n}}
$$

Therefore 

$$
\lambda_1(\Lambda)\lambda_1(\Lambda^*) \leq n [\det(\Lambda)\det(\Lambda^*)]^{\frac{1}{n}} = n
$$

using the fact that $\det(\Lambda)\det(\Lambda^*) = 1$.

The actual [theorem of Banaszczyk](ScannedPapers/BanaszczykTransferenceTheorem.pdf) is a major improvement on this bound:

**Transference Theorem**: For any full-rank Lattice $\Lambda$, the following holds

$$
1 \leq \lambda_1(\Lambda)\lambda_n(\Lambda^*) \leq n
$$

**Proof**:

**(Lower bound $1 \leq \lambda_1(\Lambda)\lambda_n(\Lambda^*)$)**: Let $\mathbf{v} \in \Lambda$ be such that $\lVert  \mathbf{v} \rVert = \lambda_1(\Lambda)$. Let $\mathbf{x}_1, \mathbf{x}_2, \cdots, \mathbf{x}_n \in \Lambda^*$ be an arbitrary set of $n$ linearly independent vectors in $\Lambda^*$. Since $\mathbf{x}_i$'s are linearly independent, not all $\langle \mathbf{x}_i, \mathbf{v} \rangle = 0$ (if they were all zero, then at least on of the $\mathbf{x}_i$ would end up being linearly dependent). Therefore, there exists an index $i$ such that $\langle \mathbf{x}_i, \mathbf{v} \rangle = z$, where $z \in \mathbb{Z} \setminus \{0\}$. 

By Cauchy-Schwarz inequality, $\lVert \langle \mathbf{x}_i, \mathbf{v} \rangle \rVert \leq  \lVert \mathbf{x}_i \rVert \cdot \lVert \mathbf{v} \rVert$, therefore $\lVert z \rVert \leq  \lVert \mathbf{x}_i \rVert \cdot \lVert \mathbf{v} \rVert$. Since the minimum value of $\lVert z \rVert$ is 1, and taking $\mathbf{x}_i = \lambda_i(\Lambda^*)$, we get $1 \leq \lambda_1(\Lambda)\lambda_n(\Lambda^*)$. \[**Note**: This proof is not entirely correct because we haven't proved that $\lambda_n(\Lambda^*)$ vector $\mathbf{x}_n$ is not orthogonal to $\mathbf{v}$. Reader's requested to fill in this gap.\]

**(Upper bound $\lambda_1(\Lambda)\lambda_n(\Lambda^*) \leq n$)**: __TODO__

## Gaussian measure on Lattices

For vectors $\mathbf{c}, \mathbf{x} \in \mathbb{R}^n$ and a real paramter $s \in \mathbb{R}$, let  

\\[
\rho_{s,\mathbf{c}}(\mathbf{x}) := e^{-\pi\frac{\lVert \mathbf{x} - \mathbf{c} \rVert}{s^2}}
\\]

be the Gaussian function centered at $\mathbf{c}$ and scaled by a factor of $s$. The total _measure_ associated to $\rho_{\mathbf{c},s}$ is $\int_{\mathbf{x} \in \mathbb{R}^n} \rho_{s,\mathbf{c}}(\mathbf{x})d\mathbf{x} = s^n$. Therefore, one can define a *continuous Gaussian distribution* around $\mathbf{c}$ with parameter $s$ by its probability density function

$$
\forall \mathbf{x} \in \mathbb{R}^n, D_{s,\mathbf{c}}(\mathbf{x}) := \frac{\rho_{s,\mathbf{c}}(\mathbf{x})}{s^n}
$$

For any discrete set $A = \{\mathbf{y}_0, \mathbf{y}_1, \mathbf{y}_2, \cdots \} \subset \mathbb{R}^n$, the Gaussian mass function can be extended to $A$ as 

$$
\rho_{s,\mathbf{c}}(A) := \sum_{\mathbf{y} \in A} \rho_{s,\mathbf{c}}(\mathbf{y})
$$

If $A$ represents the set of all lattice points of a lattice $\Lambda$, then the total measure associated with $\Lambda$ is 

$$
\rho_{s,\mathbf{c}}(\Lambda) := \sum_{y\in\Lambda} \rho_{s,\mathbf{c}}(\mathbf{y})
$$

and the **Discreate Gaussian Distibution** which is only defined on points on the lattice $\Lambda$ is 

$$
D_{\Lambda,s,\mathbf{c}}(\mathbf{x}) := \begin{cases}
\frac{\rho_{s,\mathbf{c}}(\mathbf{x})}{\rho_{s,\mathbf{c}}(\Lambda)} \text{if}\, \mathbf{x} \in \Lambda \\
0 \,\,\,\,\,\,\,\,\,\, \text{otherwise}
\end{cases}
$$

### Smoothing Parameter

**Definition**: For an $n$ dimensional lattice $\Lambda$, and a positive real $\epsilon > 0$, the smoothing parameter $\eta_{\epsilon}(\Lambda)$ is defined to be the smallest $s \in \mathbb{R}^+$, such that $\rho_{1/s}(\Lambda^* \setminus \{\mathbf{0}\}) \leq \epsilon$. 

Recall that $\rho_{1/s}(\Lambda^* \setminus \{\mathbf{0}\}) = \sum_{\mathbf{y} \in \Lambda^* \setminus \{\mathbf{0}\}} e^{-\pi s^2 \lVert \mathbf{y} \rVert}$, which is a continuous and monotonically decreasing function in $s$ such that when $s\to 0$, $\lim\rho_{1/s}(\Lambda^* \setminus \{\mathbf{0}\}) \to \infty$ and when $s\to\infty$, $\lim\rho_{1/s}(\Lambda^* \setminus \{\mathbf{0}\}) \to 0$. Therefore, $\eta_{\epsilon}(\Lambda)$ is defined for all values $\epsilon$.

**Lemma**: For an $n$ dimensional Lattice $\Lambda$ and $\epsilon=2^{-n}$, $\eta_{\epsilon}(\Lambda) \leq \sqrt{\log(1/\epsilon) + n}\cdot\lambda_1(\Lambda)$. 

**Proof**: Uses Banaszczyk theorem for proof.

**Lemma**: Let $\Lambda(\mathbf{B})$ be a lattice with basis $\mathbf{B} \in \mathbb{R}^{n\times n}$ and $\mathcal{P}(\mathbf{B})$ be the fundamental parallelopiped. For any $s \in \mathbb{R}^+$ and $\mathbf{c} \in \mathbb{R}^n$, if $D_{s,\mathbf{c}}$ is the continous Gaussian distribution, then the statistical distance between $D_{s,\mathbf{c}}\,(\text{mod}{\mathcal{P}(\mathbf{B})})$ and uniform distribution on $\mathcal{P}(\mathbf{B})$ is at most $\frac{1}{2}\rho_{1/s}(\Lambda^* \setminus \mathbf{0})$. In particular,

$$
\Delta(D_{s,\mathbf{c}}\,\,\text{mod}\,\mathcal{P}(\mathbf{B}),\, U(\mathcal{P}(\mathbf{B}))) \leq \epsilon/2
$$

**Proof**: Routine probability computation by summing probabilities at all lattice points and computing the statistical distance.

**Example [one dimensional Lattice]**: A one dimensional lattice is a set of the form $\Lambda([\tau]):=\mathbb{Z}\cdot\tau$ where $\tau \in \mathbb{R}$. The dual lattice $\Lambda^*$ consists of all points of the form $d\in \mathbb{R}$ such that $d\cdot(n\tau) \in \mathbb{Z}$. In other words $\Lambda^* = \mathbb{Z}\cdot\frac{1}{\tau}$. Therefore the smoothing parameter 

$$
\begin{aligned}
\eta_\epsilon([\tau]) &:=\min_{s > 0}\left[\sum_{y \in \mathbb{Z}\cdot\frac{1}{\tau} \setminus {0}} e^{-\pi s^2 \lVert y \rVert} \leq \epsilon\right] \\
                      & = \min_{s > 0}\left[2\sum_{k \in \mathbb{Z}_{>0}} \left (e^{-\pi\frac{s^2}{|\tau|}}\right )^k\leq \epsilon\right] \\
                      & = \min_{s > 0}\left[\left (\frac{e^{-\pi\frac{s^2}{\tau}}}{1-e^{-\pi\frac{s^2}{|\tau|}}}\right ) \leq \frac{\epsilon}{2} \right]\\
                      & = \min_{s > 0}\left[ e^{-\pi\frac{s^2}{|\tau|}} \leq \frac{\epsilon}{2+\epsilon} \right ] \\
                      & =  \min_{s > 0}\left[s \ge \sqrt{\frac{|\tau|}{\pi}\log{\frac{2+\epsilon}{\epsilon}}}\right] \\
                      & = \sqrt{\frac{|\tau|}{\pi}\log{\frac{2+\epsilon}{\epsilon}}}
\end{aligned}                  
$$ 

The fundamental parallelopiped $\mathcal{P}([\tau])$ consists of all points $t$ in the interval $[0,\tau)$. The thorem above says, that if one samples random number using the following two distributions, their statistical distance can be made arbitrarily close depending upon the choice of $\epsilon$:
1. Sample $x$ from Uniform distribution over $[0, \tau)$
2. Sample $y'$ from using Gaussian distribution with variance $s^2 > \frac{|\tau|}{\pi}\log{\frac{2+\epsilon}{\epsilon}}$ over $\mathbb{R}$, and then compute $y:=y' \mod{\tau}$, where by convention $y \in [0,\tau);\, \tau > 0$.

## Short Integer Solutions (SIS) Problem

**Definition**: The _Small Integer Solution_ (SIS) problem in $\ell_2$ norm is: Given an integer $q$, a matrix $\mathbf{A} \in (\mathbb{Z}/q\mathbb{Z})^{n\times m}$ and $\beta \in \mathbb{R}^+$, find a non-zero integer vector $\mathbf{z} \in \mathbb{Z}^m \setminus \mathbf{0}$, such that 
 1. $\mathbf{A}\mathbf{z} = \mathbf{0}^{n\times1} \mod{q}$ and 
 2. $\lVert \mathbf{z} \rVert \leq \beta$. 

This definition assumes that such a solution indeed exists. The following lemma gives a sufficient condition for this to hold.

**Lemma**: For any $q, \mathbf{A}^{n\times m}$ and $\beta \ge \sqrt{m}\cdot q^{n/m}$, the SIS-instance $(q,\mathbf{A}, \beta)$ admits a solution, i.e., there exists $\mathbf{z} \in \mathbb{Z}^m$ such that $\mathbf{A}\mathbf{z} = 0\,\,\text{mod}\,q$. (Here $q^{n/m}$ is rounded up to the nearest integer.)

**Proof**: Proof uses pigeon-hole principle. Consider all vectors $\mathbf{z} \in \mathbb{Z}^m$ whose co-ordinates are selected from the set $\{0, 1, \cdots,q^{n/m} \}$. Since each vector has dimension $m$, there are more than $(q^{n/m})^m = q^n$ such vectors. Because $\mathbf{A}\mathbf{z} \mod{q}$ has dimension $n$ and each co-ordinate of the vector can have value between $0$ and $q-1$ (because of mod $q$), there must be $\mathbf{z}_1 \neq \mathbf{z}_2$ such that $\mathbf{A}\mathbf{z}_1 = \mathbf{A}\mathbf{z}_2$. Therefore, there must exist an integer vector $\mathbf{z} = \mathbf{z}_1 - \mathbf{z}_2$ such that $\mathbf{A}\mathbf{z} = 0$. The maximum value of $\lVert \mathbf{z} \rVert$ is achieved when the coordinate-wise difference between $\mathbf{z}_1$ and $\mathbf{z}_2$ is $q^{n/m}$. Therefore, $\lVert \mathbf{z}_1 - \mathbf{z}_2 \rVert \leq \sqrt{m\times (q^{n/m})^2} = \sqrt{m}q^{n/m}$

One should treat $n$ as the security parameter, and requires $m(n) = n^{O(1)}$ and both $q(n)$ and $\beta(n)$ to be polynomially bounded. Typical values of $q$ and $m$ are such that $q^{n/m}=O(1)$ so that $\beta(n) = \tilde{O}(\sqrt{m})$.


### Solving SIS in the real-world

The SIS problem without the added constraint $\lVert \mathbf{z} \rVert \leq \beta$ is relatively simple to solve: It's a system of $n$ linear equations in $m$ variables where $m \gg n$. One trivial way to find a solution is to radomly pick $n+1$ out of $m$ column vectors from $\mathbf{A}$, call it $\mathbf{B} \in \mathbb{R}^{n\times (n+1)}$ and solve for $\mathbf{B}\mathbf{z} = $, where $z_{n+1} = -1$. If $z_i = 0$ for all $i$ (which has very low probability), pick a different random column vector and try again. Otherwise, a possible solution of $\mathbf{A}\mathbf{z} = 0$ is $[z_1, z_2, \cdots, z_{n}, -1, 0, \cdots, 0] \in \mathbf{Z}^m$. (The SISOracle sage script below provides  ```solve_without_constraint``` to achieve this.)

An alternative formulation of SIS can be seen as the lattice $\Lambda(\mathbf{A})^\perp := \{\mathbf{x} \in \mathbb{Z}^m | \mathbf{A}\mathbf{x} = \mathbf{0}\,\,\text{mod}\,q\}$. If $\mathbf{A}$ is interpreted as a linear transformation, then $x$ is in the kernel of this linear map. One can compute the basis of this linear transformation, and run it through LLL (BKZ with large blocks) to obtain short vectors. Alas! this method does not product vectors that are sufficiently shorter than reasonable values of $\beta$ (i.e, $\beta = O(\sqrt{m}))$.

[BGLS14](https://eprint.iacr.org/2014/593.pdf) provides a detailed unified framework for solving (inhomogenous)-SIS problem. TODO: Read the paper more carefully. TODO: Implement Wagner's [Generalized Birthday Problem](https://www.iacr.org/archive/crypto2002/24420288/24420288.pdf) but for mod-q!

In [95]:
def append_to_vec(v: vector, elems):
    l = v.list()
    if isinstance(elems, list):
        l.extend(elems)
    else:
        l.append(elems)
    return vector(v.base_ring(), l)
    
class SISOracle:
    def __init__(self, n : int, m : int, q : int):
        self._n    = n
        self._m    = m
        self._q    = q
        self.β = int(sqrt(float(m))*ceil(float(q)^(n/m)))+1
        self._ring = IntegerModRing(q)
        A = random_matrix(self._ring, self._n, self._m - 1)
        sis = random_vector(IntegerModRing(3), self._m-1).lift() - \
                vector(ZZ, [1 for i in range(m-1)])
        last = A * (-1*sis)
        
        # By leftover hash lemma, this martix is as good as uniform at random matrix.
        self._A = A.augment(last)
        self._sis = append_to_vec(sis,1)
        assert self._A * self._sis == vector(self._ring, 
                                             [self._ring(0) for i in range(n)])
        
    @classmethod
    def from_known_parameters(klass, A : Matrix, q : int, β : float):
        obj = klass.__new__(klass)        
        setattr(obj, '_A', A)
        setattr(obj, 'β', β)
        setattr(obj, '_q', q)
        setattr(obj, '_n', A.nrows())
        setattr(obj, '_m', A.ncols()) 
        setattr(obj, '_sis', None)
        return obj
        
    def __repr__(self):
        return "SISInstance{{ n={}, m={}, q={}, β={} }}".format(
            self._n, self._m, self._q, self.β)
        
    
    def n(self):
        return self._n
    
    def m(self):
        return self._m
    
    def q(self):
        return self._q
    
    def beta(self):
        return self.β
    
    def ring(self):
        return self._ring
    
    def A(self):
        return self._A
    
    def sis(self):
        if self._sis is not None:
            return self._sis
        else:
            return self.solve_brute_force()
    
    def _ternary_vec(self, value: int):
        result = list()
        for i in range(self.m()):
            x = (value % 3)
            value = value // 3
            if x == 2:
                result.append(-1)
            else:
                result.append(x)
        return vector(ZZ, result)
    
    def _is_zero_vec(self, vec : vector):
        return vec == vector(vec.base_ring(),
                            [0 for i in range(vec.degree())])
    
    def solve_via_lll(self):
        reduced=self._A.right_kernel_matrix().lift().BKZ(block_size=100, proof=True)
        min_v = reduced.rows()[self._n-1]
        min_norm = float(min_v.norm())
        for v in reduced.rows():
            # print("min_norm = {}, v = {}".format(min_norm, v))
            if v.norm() < min_norm:
                min_norm = float(v.norm())
                min_v = v
        return min_v
    
    def solve_without_constraint(self): 
        B=self._A.delete_columns(range(self._n,self._m))
        v=self._A.columns()[self._n]
        result = list(B.inverse()*(-1*self._A.columns()[self._n]))
        result.append(1)
        for i in range(self._n+1, self._m):
            result.append(0)
        return vector(result)
    
    def solve_brute_force(self):
        ### This uses brute force to solve SIS.
        for i in range(1, 3^self.m()):
            tv = self._ternary_vec(i)
            if self._is_zero_vec(self._A*tv):
                return tv
        raise Exception("SIS instance has large vector size")
        

#### SIS Solver without constraint example

In [25]:
sis_instance = SISOracle(5, 11, 257)
soln=sis_instance.solve_without_constraint()

assert sis_instance.A()*soln == vector([0 for i in range(sis_instance.n())])

print("A=\n{}".format(sis_instance.A()))
print("Z={}".format(soln))
print("beta = {}, soln_norm = {}".format(sis_instance.beta(),
        float(vector([int(x) for x in soln]).norm())))


A=
[ 91 251 111  22 212  71  30   9 157 213  58]
[170  38  28  27  74 163  65 182  33 149 210]
[160 152  50  22 242 195  32 214 150  42  59]
[174  79 254 164  58 172 208  33 168 172 150]
[199 108 191 132  22 134  67 191   4 109 101]
Z=(43, 143, 21, 109, 146, 1, 0, 0, 0, 0, 0)
beta = 44, soln_norm = 236.510042070099


#### SIS solver via LLL

In [68]:
sis_instance = SISOracle(5, 11, 257)
soln=sis_instance.solve_via_lll()
assert sis_instance.A()*soln == vector(sis_instance.ring(), 
                                       [0 for i in range(sis_instance.n())])
print(soln, float(soln.norm()))
print(sis_instance.sis(), float(sis_instance.sis().norm()))


(-8, -5, 2, 15, -9, 8, 18, -28, 20, -22, 30) 57.92236183029832
(0, 1, -1, -1, 1, 0, 1, 0, 1, -1, 1) 2.8284271247461903


### SIS solution via bruteforce

In [86]:
sis_instance = SISOracle(3, 5, 257)
soln=sis_instance.solve_brute_force()
assert sis_instance.A()*soln == vector(sis_instance.ring(), 
                                       [0 for i in range(sis_instance.n())])
print(soln, float(soln.norm()))
print(sis_instance.sis(), float(sis_instance.sis().norm()))

(0, -1, -1, -1, 1) 2.0
(0, -1, -1, -1, 1) 2.0


### External SIS instance

In [101]:
sis_instance = SISOracle(3, 5, 257)
sis_instance2 = SISOracle.from_known_parameters(sis_instance.A(),
                                               sis_instance.q(),
                                               sis_instance.beta())
print(sis_instance2)
print(sis_instance)

SISInstance{ n=3, m=5, q=257, β=63 }
SISInstance{ n=3, m=5, q=257, β=63 }


## Short Independent Vector Problem

**Definition** (_search_ $\text{SIVP}_\gamma$): Let $\Lambda(\mathbf{B})$ be a full rank lattice of dimension $n$. Let $\gamma(n) > 1$ be an approximation factor. Then the _search_ $\text{SIVP}_\gamma$ asks to find $n$ _linearly-independent_ lattice vectors $\mathbf{v}_1, \mathbf{v}_2,\cdots,\mathbf{v}_n \in \Lambda(\mathbf{B})$ such that $\lVert \mathbf{v}_i \rVert \leq \gamma(n)\cdot\lambda_n$.

## Overview of $SIVP$ to $SIS$ Reduction

**Problem Setting**: One is given a full-rank integer lattice $\Lambda$ with an arbitrary basis $\mathbf{B} \subset \mathbb{Z}^{n\times n}$. The $\mathtt{SIVPSolver(basis:Matrix)}$ algorithm takes $\mathbf{B}$ as its input and is expected to output $n$ linearly independent vectors $\mathbf{s}_1, \mathbf{s}_2, \cdots, \mathbf{s}_n \in \mathbb{Z}^n$ such that $\lVert \mathbf{s}_i \rVert \leq \gamma(n)\cdot\lambda_n$ for all $i \in [1,n]$.

In addition, $\mathtt{SIVPSolver}$ also has access to $\mathtt{SISOracle}$, which given a matrix $\mathbf{A} \in (\mathbb{Z}/q\mathbb{Z})^{n\times m}$  chosen uniformly at random with $m = O(n \log(q))$, finds $m$ integers $\mathbf{x} = [x_1, x_2, \cdots, x_m] \in \mathbf{Z}^m$ such that $\mathbf{A}\mathbf{x} = \mathbf{0}^n (\text{mod}\,q)$ and $\lVert \mathbf{x} \rVert \leq O(\sqrt{q})$. The input to $\mathtt{SISOracle}$ _must also satisfy_ the condition that $\mathbf{A}$ is computationally indistingushable from uniform. If $\mathbf{A}$ does not statisfy this constraint, then $\mathtt{SISOracle}$ is allowed to give wrong answers, i.e., it may **not terminate** or $\mathbf{x}$ might either not be short or $\mathbf{A}\mathbf{x} \neq \mathbf{0}^n$.) (NOTE: The $\mathtt{SISOracle}$ is expected to work for any arbitrary value of $n$ and $m = O(n)$. Also, the $n$ in $\mathtt{SISOracle}$ is different from the $n$ in $\mathtt{SIVPSolver}$.)

At a very high-level the goal of this reduction can be summarized as 

1. Given an $n\times n$ integer matrix $\mathbf{B}$, which might have come from some arbitrarily complicated distribution---even distributions that are not efficiently computable---generate a SIS instance basis $\mathbf{A}$, which has uniform distribution over $(\mathbb{Z}/q\mathbb{Z})^{u\times v}$ for some suitable choice of $q, u$ and $v$. Since the distribution of $\mathbf{B}$ is arbitrary, it includes the worst-case SIVP instances. 
2. Combine $v$ returned by the SIS oracle to find $n$ linearly-independent short vectors
3. Prove that if the SIS oracle did it's job, then SIVPSolver is guaranteed return a SIVP solution.

The high level idea behind the reduction is as follows:

1. Given the SIVP basis $\mathbf{B} \in \mathbb{Z}^{n\times n}$, sample $m$ random vectors $\mathbf{\tilde{x}}_1, \mathbf{\tilde{x}}_2, \cdots, \mathbf{\tilde{x}}_m \in \mathbb{R}^n$ from a Gaussian distribution with variance greater than the smoothing parameter $\eta_{\epsilon}(\Lambda)$. Note that one needs to know a rough estimate on $\eta_{\epsilon}(\Lambda)$ in order to run this reduction. Computing $\eta_{\epsilon}(\Lambda)$ in itself is hard, however, since the basis matirix $\mathbf{B} = [\mathbf{b}_1, \mathbf{b}_2,\cdots, \mathbf{b}_n]$ contains $n$-linearly independent vectors, 
$$
\lambda_n \leq \max_{i\in [1,\cdots,n]}\left\{\lVert\mathbf{b}_i\rVert \right\}
$$ 
and therefore 
$$
\eta_\epsilon(\Lambda) \leq \sqrt{\log(1/\epsilon) + n}\cdot \max_{i\in [1,\cdots,n]}\{\lVert\mathbf{b}_i\rVert \}
$$
Depending upon how _bad the basis vectors are_, this upper bound on $\eta_\epsilon(\Lambda)$ might be extremely frivolous. However, as long as we sample from such a Gaussian distribution, the reduction is guaranteed to go through.

2. In order to make use of the SIS oracle, we need input lattice points that have been sampled uniformly at random from some appropriate lattice. To achieve uniformly sampled lattice points, we define a new lattice $\Lambda(\mathbf{B}_q) := \Lambda(\frac{1}{q}\mathbf{B})$ which is a super-Lattice of $\Lambda(\mathbf{B})$ generated by the basis vectors $\mathbf{B}_q := [\frac{1}{q}\mathbf{b}_1, \frac{1}{q}\mathbf{b}_2,\cdots,\frac{1}{q}\mathbf{b}_n]$, and compute equivalent points $\mathbf{x}_i$ within the fundamental parallelopied $\mathcal{P}$ as 
$$
\mathbf{x}_i = \mathbf{\tilde{x}}_i\,\,\text{mod}\,\mathcal{P}(\mathbf{B})
$$
Doing this guarantees that $\mathbf{x}_i$ are uniformly distributed over $\mathcal{P}(\mathbf{B})$. Next, we need points that are uniform over $\Lambda(\mathbf{B}_q)$:

3. Since the basis $\mathbf{B}$ is linearly independent, $\mathbf{B}_q$ is also linearly independent, and $\text{span}(\mathbf{B}_q) = \text{span}(\mathbf{B}) = \mathbb{R}^n$. Therefore, each $\mathbf{x}_i$ can be written as $\mathbb{R}$-linear combination of basis vectors $\mathbf{B}_q$, such that $\mathbf{x}_i = \mathbf{B}_q\mathbf{y}_i$, where  $\mathbf{y}_i := [z_{1i} + y_{1i}, z_{2i} + y_{2i},\cdots, z_{ni} + y_{ni}]^T$, with $z_{k,i} \in \mathbb{Z}$, $y_{k,i} \in (-1/2,1/2]$ and
$$
\mathbf{x}_i = \mathbf{B}_q\begin{pmatrix}z_{1i} + y_{1i}\\ \vdots \\ z_{ni} + y_{ni}\end{pmatrix} =  \mathbf{B}_q\begin{pmatrix}z_{1i}\\ \vdots \\ z_{ni}\end{pmatrix} + \mathbf{B}_q \begin{pmatrix}y_{1i}\\ \vdots \\ y_{ni}\end{pmatrix}
$$
Therefore, the nearest lattice point $\mathbf{a}_i \in \Lambda(\mathbf{B}_q) \cap \mathcal{P}(\mathbf{B})$ that corresponds to $\mathbf{x}_i \in \mathbb{R}^n$ is $\mathbf{B}_q\begin{pmatrix}z_1\\ \vdots \\ z_n\end{pmatrix}$, where $\mathbf{z}_i := [z_{1i}, z_{2i}, \cdots, z_{ni}]$ can be computed as
$$
\mathbf{z}_i = \left \lceil \mathbf{B}_q^{-1}\mathbf{x}_i \right \rfloor = \left \lceil q\mathbf{B}^{-1}\mathbf{x}_i \right \rfloor
$$
and
$$
\mathbf{a}_i = \mathbf{B}_q\mathbf{z}_i = \frac{1}{q}\mathbf{B}\mathbf{z}_i
$$
Note that $\mathbf{a}_i$ are uniformly distributed over $\Lambda(\mathbf{B}_q) \cap \mathcal{P}(\mathbf{B})$, but we cannot feed it to the SIS oracle since it expects elements from $\mathbb{Z}/q\mathbb{Z}$. However, if $\mathbf{a}_i$ are uniformly distributed, then so are $\mathbf{z}_i$ and we can run SIS oracle on $\mathbf{z}_i$, which is what we do next.

4. Given $\{\mathbf{z}_i \in (\mathbb{Z}/q\mathbb{Z})^n \}_{i \in [1,\cdots,m]}$, which are uniformly distributed, we can ask the SIS oracle to return solutions which are smaller than $\beta = O(\sqrt{m})$. (NOTE: $m$ and $q$ should be selected in such a way that this requirement on $\beta$ can be met.) 
$$
$$
Let $\mathbf{e} = [e_1, e_2,\cdots,e_m] \in \mathbb{Z}^m$ be the response of SIS oracle, such that 
$$
\sum_{i=1}^{m}e_i\mathbf{z}_i \cong \mathbf{0}^n\,\,(\text{mod}\, q)\,\,\text{and}\,\,\lVert \mathbf{e} \rVert \leq \beta
$$
then 
$$
    \mathbf{v} = \sum_{i=1}^{m}e_i(\mathbf{\tilde{x}}_i - \mathbf{x}_i + \mathbf{a}_i) \in \Lambda(\mathbf{B})
$$
and $\lVert \mathbf{v} \rVert$ is "short."
$$
$$
**Proof**: $\mathbf{v} \in \Lambda(\mathbf{B})$. Since $\mathbf{x}_i \cong \mathbf{\tilde{x}}_i\,\,\text{mod}(\mathcal{P})$, $\mathbf{\tilde{x}}_i - \mathbf{x}_i$ is an element of $\Lambda(\mathbf{B})$. Since $e_i \in \mathbb{Z}$, $e_i(\mathbf{\tilde{x}}_i - \mathbf{x}_i) \in \Lambda(\mathbf{B})$, therefore $\sum_{i=1}^{m}e_i(\mathbf{\tilde{x}}_i - \mathbf{x}_i) \in \Lambda(\mathbf{B})$. Since, $\sum_{i=1}^{m}e_i\mathbf{z}_i = \mathbf{0}^n\,\,\text{mod}\,q\,\implies \frac{1}{q}\mathbf{B}\sum_{i=1}^{m}e_i\mathbf{z}_i = \mathbf{0}^n\,\,\text{mod}\,q$, $\implies$ $\frac{1}{q}\sum_{i=1}^{m}e_i\mathbf{B}\mathbf{z}_i = \mathbf{0}^n\,\,\text{mod}\,q$, $\implies$ $\frac{1}{q}\sum_{i=1}^{m}e_i\mathbf{a}_i = \mathbf{0}^n\,\,\text{mod}\,q$. Therefore, $\sum_{i=1}^{m}e_i\mathbf{a}_i$ must be a $q$ multiple of $\Lambda(\mathbf{B}_q)$ and therefore an element of $\Lambda(\mathbf{B})$.
$$
$$
**Proof**: $\mathbf{v}$ is short. Since $\mathbf{v} = \sum_{i=1}^{m}e_i(\mathbf{\tilde{x}}_i - \mathbf{x}_i + \mathbf{a}_i)$, therefore
$$
\begin{aligned}
\lVert \mathbf{v}  \rVert &\leq \left \lVert \sum_{i=1}^{m}e_i\mathbf{\tilde{x}}_i \right \rVert + \left  \lVert \sum_{i=1}^{m}e_i(\mathbf{x}_i - \mathbf{a}_i) \right \rVert \\
 &\leq \lVert \mathbf{e} \rVert \cdot \lVert \mathbf{\tilde{x}} \rVert + \frac{n\max_i\lVert \mathbf{b}_i\rVert}{q} \\
 & \leq \beta\cdot s\sqrt{n} +  \frac{n\max_i\lVert \mathbf{b}_i\rVert}{q}
\end{aligned}
$$
where we are using the fact that $\lVert\mathbf{e}\rVert \leq \beta$ and the average length of Gaussian is $s\sqrt{n}$.
$$
$$
Note that if $\mathbf{b}_i$ are large, then the above vector is not guaranteed to be small! However, if one choose $q > n^2$, then the resulting vector will be smaller than all $\mathbf{b}_i$, and one could use the new vector as a basis element and re-rerun the reduction.
$$
$$
One question still unanswered is, why can't $\mathbf{v}$ be $\mathbf{0}^n$? Answer: randomness!

The SIVPSolver below implements all these steps:

In [38]:
class SIVPSolver:
    
    ## Bikeshed much???
    class SISInputInstance:
        def __init__(self, q : int, 
                     Bq : Matrix, 
                     x_tilde : Matrix, 
                     x: Matrix, 
                     z : Matrix, 
                     a : Matrix):
            self._q = q
            self._Bq = Bq
            self._x_tilde = x_tilde
            self._x = x
            self._a = a
            self._z = z
            
        def Bq(self):
            return self._Bq
        
        def x_tilde(self):
            return self._x_tilde
        
        def x(self):
            return self._x
        
        def z(self):
            return self._z
        
        def a(self):
            return self._a
        
        def __repr__(self):
            return "q = {}\nBq=\n{}\nx_tilde=\n{}\nx=\n{}\nz=\n{}\na=\n{}\n".format(
                self._q, self._Bq, self._x_tilde, self._x, self._z, self._a
            )
    
    ## =========== end of class SISInputInstance ==============
    
    def __init__(self, B : Matrix):
        assert B.nrows() == B.ncols()
        if B.base_ring() == ZZ:
            B=B.change_ring(RR)
        
        self._B = B
        self._Binv = B.inverse()
        self._n = B.nrows()
        self._q = random_prime(self._n * self._n * self._n, lbound=self._n * self._n)
        self._m = self._n*int(log(2)/(log(1.125)*log(self._q)))
        self._eta_sq = self._estimate_eta_sq()
        self._gauss = RealDistribution('gaussian', sqrt(self._eta_sq))
    
    def solve(self):
        A = self.generate_sis_samples()
        oracle = SISOracle.from_known_parameters(A, self._q, sqrt(self._m))
    
    def generate_sis_samples(self):
        ## See the discription in the cell above for the meaning of these items
        x_tilde_list = list() 
        x_list       = list()
        z_list       = list()
        a_list       = list()
        
        for i in range(self._m):
            x_tilde = self._sample_continuous_gaussian()
            x_tilde_list.append(x_tilde)
            
        x_tilde_list = Matrix(RR, x_tilde_list).transpose()
            
        for x_tilde in x_tilde_list.columns():
            x = self._reduce_mod_parallelopiped(x_tilde)
            x_list.append(x)
            z = self._quantize_to_q_grid(x)
            z_list.append(z)
        x_list = Matrix(RR, x_list).transpose()
        z_list = Matrix(Zmod(self._q), z_list).transpose()
        a_list = ((self._B*z_list.lift())/self._q)
            
        return SIVPSolver.SISInputInstance( self._q,
                                self._B/self._q,
                                x_tilde_list,
                                x_list,
                                z_list,
                                a_list)
    
    def _estimate_eta_sq(self, epsilon = 2^-128):
        lambda_n_sq = 0
        
        for x in self._B.columns():
            n = x * x
            if n > lambda_n_sq:
                lambda_n_sq = n
        assert lambda_n_sq != 0
        
        return (float(log(1/epsilon)/log(2)) + self._n)*lambda_n_sq
    
    def _reduce_mod_parallelopiped(self, v : vector):
        assert v.base_ring() == RR
        vp = self._Binv*v
        vreduced = vector(RR, [ x % 1 for x in vp])
        return self._B * vreduced
    
    def _quantize_to_q_grid(self, v : vector):
        vp = self._q * (self._Binv * v)
        z_list = list()
        for c in vp:
            z = int(c)
            if c - z > 0.5:
                z = z + 1
            assert z <= self._q
            z_list.append(z)
            
        return vector(Zmod(self._q), z_list)
    
    def _sample_continuous_gaussian(self):
        return [self._gauss.get_random_element() for _ in range(self._n)]
    
    def __repr__(self):
        return "B=\n{}\nn={}\nη={}\nq={}\nm={}".format(
            self._B, self._n, float(sqrt(self._eta_sq)), 
            self._q, self._m
        )
    
B=random_matrix(ZZ, 3,3)
while B.det() == 0:
    B = random_matrix(ZZ, 3,3)

sivp_solver = SIVPSolver(B); print(sivp_solver)
sivp_solver.solve()



B=
[-2.00000000000000 -25.0000000000000 -1.00000000000000]
[ 1.00000000000000 0.000000000000000  1.00000000000000]
[-5.00000000000000 0.000000000000000  1.00000000000000]
n=3
η=286.1380785564899
q=13
m=6


NameError: name 'SISOracle' is not defined